In [4]:
import os
import random
import numpy as np
import pandas as pd

from itertools import combinations

In [5]:
import random

NUM_PAIRS = 100000
POSITIVE_PORTION = 0.5

IDENTITIES_FOLDER = "/gpfs/gpfs0/r.karimov/IJB/aligned_data_for_fusion/debug"
RESULT_META_DIR = "/gpfs/gpfs0/r.karimov/IJB/aligned_data_for_fusion/metadata_refuse_verification"

In [6]:
def randomized_round(number):
    return int(number) + int(random.random() < (number % 1))


def sample_pairs_from_directory(prefix, directory, n_pairs):
    pairs = list(combinations(os.listdir(os.path.join(prefix, directory)), r=2))
    n_sampled_pairs = n_pairs if n_pairs < len(pairs) else len(pairs)
    pairs = [tuple(map(lambda x: os.path.join(directory, x), pair)) for pair in pairs]
    return random.sample(pairs, n_sampled_pairs)


def generate_positive_pairs(identities_dir, n_pairs):

    remaining_directories = sum(1 for _ in os.listdir(identities_dir))
    remaining_pairs = n_pairs

    positive_pairs = []

    for dir in os.listdir(identities_dir):
        needed_pairs_amount = randomized_round(remaining_pairs / remaining_directories)
        positive_pairs.extend(sample_pairs_from_directory(identities_dir, dir, needed_pairs_amount))

    return positive_pairs


def generate_negative_pairs(identities_dir, n_pairs):
    negative_pairs = set()
    identities = list(os.listdir(identities_dir))
    while len(negative_pairs) < n_pairs:
        left, right = random.sample(identities, 2)
        left = os.path.join(left, random.choice(os.listdir(
            os.path.join(identities_dir, left))))
        right = os.path.join(right, random.choice(
            os.listdir(os.path.join(identities_dir, right))))
        negative_pairs.add((left, right,))

    return list(negative_pairs)

In [7]:
# generate_negative_pairs(IDENTITIES_FOLDER, int(NUM_PAIRS * (1 - POSITIVE_PORTION)))

# generate_positive_pairs(IDENTITIES_FOLDER, int(NUM_PAIRS * POSITIVE_PORTION))

In [ ]:
meta_path = os.path.join(RESULT_META_DIR, "pairs_" + str(NUM_PAIRS) + "_prob_" + str(POSITIVE_PORTION) + ".csv")
with open(meta_path, "w") as f:
    for pair in generate_positive_pairs(
            IDENTITIES_FOLDER, int(NUM_PAIRS * POSITIVE_PORTION)):
        f.write(",".join((pair[0], pair[1], "1")) + "\n")
    for pair in generate_negative_pairs(
            IDENTITIES_FOLDER, int(NUM_PAIRS * (1 - POSITIVE_PORTION))):
        f.write(",".join((pair[0], pair[1], "0")) + "\n")